# lecure du zip et affichage des candles, 
# avec superposition de marques 
# suivant un calcul fait sur les candles

avec un yield et quelques fonctions, on parametre le svm

Voila le resultat 



In [ ]:
le meme programme avec un autre plot des detections mis dans zlibplot01

##### 

# ici on trace en plus les plots des patterns detectes
#

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.colors import colorConverter
import matplotlib.pyplot as plt
import numpy as np


paire="EURUSD"
annee=2014
mois=4

nomfich=".\\data\\HISTDATA_COM_ASCII_{0}_M1{1:4d}{2:02d}.zip".format(paire,annee,mois)
#nomfich2="c:\\data\\{0}_M1_{1:4d}{2:02d}.csv".format(paire,annee,mois)
nomfichshort="DAT_ASCII_{0}_M1_{1:4d}{2:02d}.csv".format(paire,annee,mois)

print nomfich

lstdata=[-1,0,0,0,0]

#------------------------------------------------------------------------------
#lecture du zip en coroutine
#------------------------------------------------------------------------------
def scanfile(debut,longueur):
    import zipfile    
    fh = open(nomfich, 'rb')
    z = zipfile.ZipFile(fh) # classe lisant le zipdanzs le fichier ouvert
    
    minute =0
    encore =0
    
    quadvalues=[]
    
    with z.open(nomfichshort) as zz:
        for eachLine in zz:
            encore = 1
            minute = minute+1
            if debut > minute :
                continue
            
            splitLine = eachLine.split(";")
            #lu open =1 high =2 low=3 close=4
            #quad genere open close hight low (1,4,2,3)
            quadvalues = [minute,float(splitLine[1]),float(splitLine[4]),float(splitLine[2]),float(splitLine[3])]
            
            print quadvalues
            
            if minute == debut+longueur : #on a depasse la derniere valeur a lire : encore=0
                encore =0
                z.close()
                fh.close()
        
            yield encore,quadvalues
    
    #fin de la boucle car plus rien a lire
    encore =0
    z.close()
    fh.close()
    yield encore,[] #liste vide
           

#--------------------------------------------
## dessine des courbes des signaux a et b
#--------------------------------------------
def cc(arg):
    return colorConverter.to_rgba(arg, alpha=0.6)

#--------------------------------------------
#dessine le waterfall des yes et des no
#--------------------------------------------
def multidraw(courbesA,courbesB):

    fig = plt.figure()
    ax = fig.gca(projection='3d')

    verts = []
    facecolor=[]
    for lacourbe in courbesA:
        xs=[]
        closepoly=[]
        for idx,X in enumerate(lacourbe):
            xs.append(idx)
            closepoly.append(0)
            xs.append(idx)
            closepoly.append(X)
            xs.append(idx+.3)
            closepoly.append(X)
            xs.append(idx+.3)
            closepoly.append(0)
        xs.append(0)
        closepoly.append(0)
            
        verts.append(list(zip(xs, closepoly)))
        facecolor.append(cc('b'))

    for lacourbe in courbesB:
        xs=[]
        closepoly=[]
        for idx,X in enumerate(lacourbe):
            xs.append(idx)
            closepoly.append(0)
            xs.append(idx)
            closepoly.append(X)
            xs.append(idx+.3)
            closepoly.append(X)
            xs.append(idx+.3)
            closepoly.append(0)
        xs.append(0)
        closepoly.append(0)
        
        verts.append(list(zip(xs, closepoly)))
        facecolor.append(cc('y'))
    
    print len(facecolor) ,len(verts)
     
    poly = PolyCollection(verts, facecolors=facecolor)
    zs = range(0,len(facecolor))
    poly.set_alpha(0.7)
    ax.add_collection3d(poly, zs=zs, zdir='y')
    
    ax.set_xlabel('X')
    ax.set_xlim3d(0,5)
    ax.set_ylabel('Y')
    ax.set_ylim3d(-1, len(verts))
    ax.set_zlabel('Z')
    ax.set_zlim3d(-3, 3)

    plt.show()


        

#------------------------------------------------------------------------------
#dit si une cande monte ou descend
#------------------------------------------------------------------------------
def senscandle(data):
    
    if data[1]<data[2]:
        return 1   #deb < fin : ca monte
    if data[1]>data[2] :
        return 2  #deb > fin  ; ca descend
    return 0  #deb = fin


stackpast=[[-1,0,0,0,0]]




#------------------------------------------------------------------------------
#detecte les retournements montee descente
#la detection se fait sur les 2 eme a xieme items de la stack
# car le dernier item est celui qui correspond a l'achat/vente
#------------------------------------------------------------------------------
def testrevertupdown(stackdata):
    stackdata.insert(0,['date','deb','fin','top','low'])
    rdata = map(list,zip(*stackdata))
    #[list(i) for i in zip(*stackdata)] ferait aussi la transposee
    stack = {d[0]:d[1:] for d in rdata}  #on en fait un dict

    #test une descente apres deux montees nettes : signal de depart
    
    #le dernier commence plus bas que finit le prec
    cond1 = (stack['fin'][-2] > stack['deb'][-1]) 
    # le precedent etait parti plus bas que la fin de celui d'avant et etait descendu
    # debut < fin pour le precedent
    cond2 = (stack['fin'][-3] > stack['deb'][-2]) and  (senscandle(stackdata[-2]) ==2)    
    #celui d'avant etait une montee : commence + haut que son prec et est une montee
    #celui encore avant : idem
    cond3 = (stack['fin'][-4] < stack['deb'][-3]) and (senscandle(stackdata[-3]) ==1) 
    cond4 = (stack['fin'][-5] < stack['deb'][-4]) and  (senscandle(stackdata[-4]) ==1)
    
    return cond1 , cond2 and cond3 and cond4 

#---------------------------------
#normlise les dnnees du pattern
#---------------------------------
def normalisedata(stackdata):
    valmin = float(stackdata[-5][2])
    valmax = float(stackdata[-2][2])
    scale = (valmax - valmin) #/(valmax+valmin)*2
    output=[]
    for i in [-5,-4,-3,-2,-1]:
            output.append((stackdata[i][2]-valmin)/scale)
    return output

#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

def extractfeatures(stackdata):
    stackdata.insert(0,['date','deb','fin','top','low'])
    rdata = map(list,zip(*stackdata))
    #[list(i) for i in zip(*stackdata)] ferait aussi la transposee
    rdict = {d[0]:d[1:] for d in rdata}  #on en fait un dict

    featureline=[]
    #sort les features d'avan le retournement
    featureline.append(rdict['low'][-2] - rdict['top'][-2]) #delta minmax
    featureline.append(rdict['deb'][-2] - rdict['fin'][-2]) #delta deb fin
    featureline.append(rdict['low'][-3] - rdict['top'][-3]) #delta minmax
    featureline.append(rdict['deb'][-3] - rdict['fin'][-3]) #delta deb fin
    featureline.append(rdict['low'][-4] - rdict['top'][-4]) #delta minmax
    featureline.append(rdict['deb'][-4] - rdict['fin'][-4]) #delta deb fin
    featureline.append(rdict['low'][-5] - rdict['top'][-5]) #delta minmax
    featureline.append(rdict['deb'][-5] - rdict['fin'][-5]) #delta deb fin
    
#    print "feature",stackdata,featureline
    return [i * 10000 for i in featureline] #muliply par 10000

#------------------------------------------------------------------------------
#un petit essai d'analyse triviale pour prenre une decision triviale
# on met 1 si on sort rentre plus bas qu'on est sorti avant
# retourne ladifff qui vaut 1,0 ou 1 suivant que detection a ete faite ou non
# ok qui vaut 0 tant qu'on n'a âs eu assez de donnees
# learning qui contient une ligne de learnig matrice (feature  groupe)
#------------------------------------------------------------------------------

def analyse(data):
    global lstdata,stackpast
    ladiff =0
    ok=0
    learning=[] #init bidon
    
    stackpast.append(data[:]) #ajout de la derniere valeur ds stackpast
    
    if len(stackpast)==7 :
        ok=1
        del stackpast[0]  #stackpast ne doit plus s'allonger   
        yess,detected = testrevertupdown(stackpast)
        if detected:
            if yess:
                ladiff =1
            else:
                ladiff = -1
                    
        learning=extractfeatures(stackpast)  #extraction des features
        learning.extend([ladiff]) # categorie en fin de feature
        
    return ladiff,ok,learning    


#------------------------------------------------------------------------------
#plot le graph des candle de candlegraf
# et met les points de analysegraf qui contient des doublets (index,valeur) des marques a rajouter a candlegraf
#------------------------------------------------------------------------------

def plotresu(candlegraf,analysetab):
    #print entrybar
    import matplotlib.pyplot as ml

    from matplotlib.finance import candlestick
    from matplotlib.finance import candlestick_ohlc
    import matplotlib.pyplot as plt
    
    lafig = plt.figure()
    ax1 = plt.subplot(1,1,1)
    #ax2 = plt.subplot(2,1,2)


    tabplot=[]
    X=[]
    Y=[]


    #print tabplot
    #candlestick open high low close
    candlestick_ohlc(ax1,candlegraf, width=.75, colorup='r', colordown='g')
    #yscale = ax1.get_ybound()  #renvoie le min et max de l'afficahge
    #print yscale

    for i in range(len(analysetab)):
        if analysetab[i][0]==1 :
            indx = analysetab[i][1]
            X.append(candlegraf[indx][0])
            #on ajoute a y le doublet yscale multiplie par 1 (=lui meme) ou 0 (vide) 
            #c'est pas ce que je voulais faire , je voulais multipler chaque point de y (des 1) par une valeur
            #mais bon ca marche un peu. il va falloir passer tout ca en traiteent listes/matrices sinon c'est moche
            Y.append([candlegraf[indx][1],candlegraf[indx][2],candlegraf[indx][3],candlegraf[indx][4]])  

    print len(X), len(Y)

    ax1.plot(X,Y,'bo')

    X=[]
    Y=[]
    for i in range(len(analysetab)):
        if analysetab[i][0]==-1 :
            indx = analysetab[i][1]
            X.append(candlegraf[indx][0])#on ajoute a y le doublet yscale multiplie par 1 (=lui meme) ou 0 (vide) 
            #c'est pas ce que je voulais faire , je voulais multipler chaque point de y (des 1) par une valeur
            #mais bon ca marche un peu. il va falloir passer tout ca en traiteent listes/matrices sinon c'est moche
            Y.append([candlegraf[indx][1],candlegraf[indx][2],candlegraf[indx][3],candlegraf[indx][4]])  

    ax1.plot(X,Y,'yo')

    plt.show()


#------------------------------------------------------------------------------
# analyse des resultats en comparant les resultats de la matrice des resultats
# learnresu contient en 0 reel 1 estimé
#------------------------------------------------------------------------------

def checkquality(learnresu):
    nbyesifyes=0
    nbyesifno=0
    nbnoifyes=0
    print "learnresu size",len(learnresu)
    for i in learnresu:
        valestim = i[1]
        valreelle = i[0]
        #print valreelle,valestim
    
        if valestim == 1 and valreelle==1:
            nbyesifyes = nbyesifyes+1
            #print "yesIFyes=",nbyesifyes
    
        if valestim == 1 and valreelle==-1:
            nbyesifno = nbyesifno+1
            #print "yesIFno=",nbyesifno
  
        if valestim == -1 and valreelle==1:
            nbnoifyes = nbnoifyes+1
            #print "noIFyes=",nbnoifyes
    
    print '%correct',float(nbyesifyes)/float(nbyesifyes+nbyesifno) *100.0
    print '%detected' , float(nbyesifyes)/float(nbyesifyes+nbnoifyes) *100.0 


#------------------------  MAIN  ---------------------------------
datebar=[]
candlegraf=[]
analysetab=[]
learnmat=[]

minute =0
index=0

lescanfile = scanfile(1000,1000)
graphyes = []
graphno = []

while True:
    minute = minute+1
    encore,quadvalues = lescanfile.next()
    if encore ==0:
        break
            
    resu,ok,features = analyse(quadvalues)    
    if  resu<>0 :  #pile assez grosse pour faire les features : on genere les matrices
        learnmat.append(features) #matrice des features learning
        analysetab.append([resu, index]) #liste des events detectes pour la viualisation des detections 
        if (resu ==1):
            graphyes.append(normalisedata(stackpast))
        else:
            graphno.append(normalisedata(stackpast))
 
    if ok:
        #ordre ohlc
        candlegraf.append(quadvalues) #matrice des valeurs brutes pour le graph cndles
        index= index+1

#icci on a la matrice de learning qui a son ensemble d'apprentissage et ses evenements en sortie
print "learning size = ",len(candlegraf)
print "positve size =",len([x for x in analysetab if x[0]==1])
print "negative size =",len([x for x in analysetab if x[0]==-1])

#on dessine les resultats calcules
plotresu(candlegraf,analysetab)

#on fait la waterfall des patterns trouves
multidraw(graphyes,graphno)

#on va faire les estimations

#on a la matrice de features
#print learnmat
features = [x[0:-2] for x in learnmat]
categories = [x[-1] for x in learnmat]

from sklearn import svm

#clf = svm.SVC(gamma=0.001, C=100.)
clf = svm.SVC(C=1, cache_size=200, class_weight=None, coef0=0.0, degree=2,
              gamma=0.0, kernel='rbf', max_iter=-1, probability=False, random_state=None,
              shrinking=True, tol=0.001, verbose=False)
clf.fit(features, categories)

learnresu =[]

#on verifie l'efficacite du learning en recalculant les resultats estimes sur les vleurs de check
nbyesifyes = 0
nbnoifyes = 0
nbyesifno = 0

#on rescane l'ensemble de check
lescanfile = scanfile(100,2000)

while True:
    encore,quadvalues = lescanfile.next()
    if encore ==0:
        break

    resu,ok,features = analyse(quadvalues)     #calcul des features
    if  resu<>0 :  #on a un resultat
        valout = clf.predict(features[:-2])
        learnresu.append([resu,valout[0]]) #matrice valeur calc, valeur reelle            
    
    if ok:
        index= index+1

checkquality(learnresu)

     


.\data\HISTDATA_COM_ASCII_EURUSD_M1201404.zip
[1000, 1.37902, 1.37906, 1.37906, 1.37902]
[1001, 1.37905, 1.37911, 1.37912, 1.37905]
[1002, 1.37912, 1.3792, 1.3792, 1.37912]
[1003, 1.37919, 1.37928, 1.37929, 1.37919]
[1004, 1.37927, 1.3793, 1.37931, 1.37927]
[1005, 1.37929, 1.37928, 1.3793, 1.37928]
[1006, 1.37927, 1.37918, 1.37927, 1.37918]
[1007, 1.3792, 1.37923, 1.37923, 1.3792]
[1008, 1.37921, 1.37917, 1.37921, 1.37917]
[1009, 1.37914, 1.37914, 1.37914, 1.37914]
[1010, 1.37913, 1.37912, 1.37914, 1.37912]
[1011, 1.37911, 1.37911, 1.37911, 1.37908]
[1012, 1.37909, 1.3791, 1.37912, 1.37909]
[1013, 1.37911, 1.37911, 1.37911, 1.3791]
[1014, 1.37912, 1.37922, 1.37924, 1.37912]
[1015, 1.37919, 1.37921, 1.37925, 1.37919]
[1016, 1.3792, 1.3792, 1.37924, 1.3792]
[1017, 1.37921, 1.37921, 1.37921, 1.37921]
[1018, 1.37922, 1.37922, 1.37923, 1.37922]
[1019, 1.37924, 1.37923, 1.37924, 1.37921]
[1020, 1.37922, 1.37919, 1.37922, 1.37919]
[1021, 1.3792, 1.37919, 1.3792, 1.37919]
[1022, 1.37921, 1.379

ImportError: No module named sklearn